# Introduction

To address options for portability of PERCEIVE Hadoop Ecosystem and apply big data technology for PERCEIVE project, this notebook aims to build an Apache Nifi + Kite sdk + Apache Spark pipeline, and show the potential value and usage of this data pipeline.

This article is consisted of three parts. For the first part, we will setup the environment and relevant components, and build dependency between each component. The second part will show a simple Nifi data flow from local and the web to HDFS that demonstrates several fundamental capabilities of Nifi, including:


**(need to list the capabilities of nifi from the end example)**

The third part will address the advantage of Nifi and show why the pipeline can potentially contribute to PERCEIVE project in the future.

# Overview

Before moving to next part, here is a brief introduction about Apache Nifi and Kite sdk.

a. Apache NiFi:

Nifi is an easy to use, powerful, and reliable dataflow tool that enables the automation of data flow between systems.Some of the use cases include, but are not limited to:

- **Big Data Ingest**– Offers a simple, reliable and secure way to collect data streams.
- **IoAT Optimization**– Allows organizations to overcome real world constraints such as limited or expensive bandwidth while ensuring data quality and reliability.
- **Compliance**– Enables organizations to understand everything that happens to data in motion from its creation to its final resting place, which is particularly important for regulated industries that must retain and report on chain of custody.
- **Digital Security**– Helps organizations collect large volumes of data from many sources and prioritize which data is brought back for analysis first, a critical capability given the time sensitivity of identifying security breaches

![egauge_portfolio](img/nifi_intro.png)

b. Apache Kite sdk:

From the official website of kite sdk, Kite is a high-level data layer for Hadoop, which provides a high-level API working with Hive, Oozie, HDFS and HBase. Using Kite, it is much convenient to maintain datasets and relevant metadata through Avro schema. 

Although most jobs are implemented through the Kite Command Line Interface, Nifi provides the following kite processors that realize the Kite function: ConvertAvroSchema, ConvertCSVToAvro, ConvertJSONToAvro, InferAvroSchema, StoreInKiteDataset.

Here is the example of a schema:

![egauge_portfolio](img/kite_schema.png)

## 1. Setting up the environment

### 1.1 Installation
This tutorial was tested using the following environment and components:

(Since the installation for the above components can easily be found through google, we will simply put the link here)

- Ubuntu(64 bit) on VirualBox:
    - https://linus.nci.nih.gov/bdge/installUbuntu.html
- Apache Nifi 1.3.0:
    - For installation: https://www.youtube.com/watch?v=ZTXRm7tqQs4
    - User guide: https://nifi.apache.org/docs.html
    - Required jdk environment
- Apache Hadoop 2.8.1 + Apache Spark 2.2.0 + Scala 2.12.3:
    - Need to change the version: https://medium.com/@ivanermilov/setting-up-hadoop-spark-hive-development-environment-on-ubuntu-94f0f8166ef1
- Apache Kite sdk (in progress)
    - http://kitesdk.org/docs/1.1.0/Install-Kite.html
    - http://kitesdk.org/docs/1.0.0/Kite-SDK-Guide.html

### 1.2 Building the dependency

